In [12]:
%pip install pandas
import pandas as pd
import ast

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
df = pd.read_csv('./dataset/rest_dataset.csv')


In [ ]:
def extract_last_location(loc_str):
    try:
        loc_list = ast.literal_eval(loc_str)
        return loc_list[-1] if loc_list else None
    except (ValueError, SyntaxError):
        return None

df['location'] = df['original_location'].apply(extract_last_location)

df['cuisine'] = df['cuisines'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)

def extract_certificate_of_excellence(award_str):
    if isinstance(award_str, str):
        index = award_str.find('Certificate of Excellence')
        if index != -1:  # Si encontramos 'Certificate of Excellence'
            return award_str[index:index+30]  # 'Certificate of Excellence' tiene 26 caracteres
    return None

df['last_certificate_excellence'] = df['awards'].apply(lambda x: extract_certificate_of_excellence(x) if isinstance(x, str) else None)

df['top_tag'] = df['top_tags'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else None)

columns_to_keep = [
    'price_level', 'vegan_options', 'gluten_free', 
    'open_days_per_week', 'avg_rating', 'total_reviews_count', 
    'food', 'service', 'atmosphere', 'excellent', 'location', 
    'award', 'cuisine', 'top_tag'
]

df_filtered = df[columns_to_keep]

df_filtered.to_csv('./dataset/cleaned.csv', index=False)


In [18]:
cuisine_unique_values = df['cuisine'].unique()
award_unique_values = df['award'].unique()
top_tag_unique_values = df['top_tag'].unique()

print(f"Valores únicos en 'cuisine': {len(cuisine_unique_values)} valores diferentes")
print(cuisine_unique_values)

print(f"\nValores únicos en 'award': {len(award_unique_values)} valores diferentes")
print(award_unique_values)

print(f"\nValores únicos en 'top_tag': {len(top_tag_unique_values)} valores diferentes")
print(top_tag_unique_values)


Valores únicos en 'cuisine': 66 valores diferentes
['Steakhouse' 'Seafood' 'Mediterranean' 'Cafe' 'Spanish' 'Mexican'
 'Fast food' 'Italian' 'European' 'Barbecue' 'Japanese' 'Deli' 'American'
 'French' 'Pizza' 'International' 'Sushi' None 'Healthy' 'Bar' 'Chinese'
 'Lebanese' 'African' 'German' 'Polish' 'Chilean' 'Brew Pub' 'Latin'
 'British' 'Indian' 'Peruvian' 'Asian' 'Catalan' 'Thai' 'Russian'
 'Caribbean' 'Greek' 'Wine Bar' 'Brazilian' 'Philippine' 'Grill' 'Fusion'
 'Hawaiian' 'Contemporary' 'Argentinian' 'Venezuelan' 'Diner' 'Pub'
 'Middle Eastern' 'Arabic' 'Irish' 'Gastropub' 'Dutch' 'Moroccan'
 'Hungarian' 'Turkish' 'Swiss' 'Central American' 'Nepalese'
 'Southern-Italian' 'South American' 'Colombian' 'Cuban' 'Soups' 'Tibetan'
 'Korean']

Valores únicos en 'award': 1 valores diferentes
[None]

Valores únicos en 'top_tag': 3 valores diferentes
['Mid-range' 'Cheap Eats' 'Fine Dining']
